# Causal Forest - HTE

In [ ]:
import econml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

: 

## Simulate Data

In [1]:
np.random.seed(42)
n = 1000

# Features
age = np.random.normal(40, 10, n)
engagement = np.random.uniform(0, 1, n)
X = np.column_stack([age, engagement])

# Treatment assignment (random)
T = np.random.binomial(1, 0.5, n)

# True treatment effect varies by engagement
true_te = 2 + 3 * engagement
Y = true_te * T + age * 0.1 + np.random.normal(0, 1, n)

df = pd.DataFrame({'age': age, 'engagement': engagement, 'T': T, 'Y': Y})


NameError: name 'np' is not defined